In [90]:
import pandas as pd

In [91]:
data = pd.read_csv("/content/100_Unique_QA_Dataset.csv")

In [92]:
data.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [93]:
def tokenizer(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [94]:
tokenizer("My name' is Nikhil?")

['my', 'name', 'is', 'nikhil']

In [95]:
vocab = {'UNK' : 0}
def vocabulary(column):
  quention_token = tokenizer(column['question'])
  answer_token = tokenizer(column['answer'])
  merged_token = quention_token + answer_token
  for token in merged_token:
    if token not in vocab:
      vocab[token] = len(vocab)

In [96]:
data.apply(vocabulary, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [97]:
len(vocab)

324

In [98]:
def word_to_id(text, vocab):
  id_text =[]
  for token in tokenizer(text):
    if token in vocab:
      id_text.append(vocab[token])
    else:
      id_text.append(vocab['UNK'])
  return id_text

In [99]:
import torch
from torch.utils.data import Dataset, DataLoader

In [100]:
class Customdataset(Dataset):
  def __init__(self, data, vocab):
    self.data = data
    self.vocab = vocab

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    num_question = word_to_id(self.data.iloc[idx]['question'], self.vocab)
    num_answer = word_to_id(self.data.iloc[idx]['answer'], self.vocab)

    return torch.tensor(num_question), torch.tensor(num_answer)


In [101]:
dataset = Customdataset(data, vocab)

In [102]:
data_loder = DataLoader(dataset, batch_size=1, shuffle=True)

In [103]:
for question, answer in data_loder:
  print(question, answer[0])

tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([188])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([205])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([85])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([116])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([254])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([199])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([85])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([321])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[ 42, 107,   2, 108,  19, 10

In [104]:
from typing_extensions import final
import torch.nn as nn
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, x):
    embedded_question = self.embedding(x)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output


In [105]:
model = SimpleRNN(len(vocab))

In [106]:
learning_rate = 0.001
epochs = 20

In [107]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [108]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in data_loder:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()
  avg_loss = total_loss / len(data_loder)

  print(f"Epoch: {epoch+1}, Loss: {avg_loss:4f}")

Epoch: 1, Loss: 5.825152
Epoch: 2, Loss: 5.073490
Epoch: 3, Loss: 4.172816
Epoch: 4, Loss: 3.474569
Epoch: 5, Loss: 2.911758
Epoch: 6, Loss: 2.382061
Epoch: 7, Loss: 1.902316
Epoch: 8, Loss: 1.489662
Epoch: 9, Loss: 1.148685
Epoch: 10, Loss: 0.882181
Epoch: 11, Loss: 0.684931
Epoch: 12, Loss: 0.539258
Epoch: 13, Loss: 0.435774
Epoch: 14, Loss: 0.352168
Epoch: 15, Loss: 0.292881
Epoch: 16, Loss: 0.247372
Epoch: 17, Loss: 0.208058
Epoch: 18, Loss: 0.176364
Epoch: 19, Loss: 0.152338
Epoch: 20, Loss: 0.132162


In [109]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = word_to_id(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [110]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [111]:
predict(model, "What is the largest planet in my system?")

I don't know
george-washington


In [112]:
predict(model, "What is the capital of Germany?")

berlin


In [113]:
list(vocab.keys())[7]

'paris'